#import library

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
import json
import numpy as np
import ssl
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

#Web scrape Bangkok District data from Wikipedia

In [30]:
ssl._create_default_https_context = ssl._create_unverified_context
url = 'https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok'
df = pd.read_html(url)[0]
df.head()

,District(Khet),MapNr,Post-code,Thai,Popu-lation,No. ofSubdis-trictsKhwaeng,Latitude,Longitude
0,Bang Bon,50,10150,บางบอน,105161,4,13.659200,100.399100
1,Bang Kapi,6,10240,บางกะปิ,148465,2,13.765833,100.647778
2,Bang Khae,40,10160,บางแค,191781,4,13.696111,100.409444
3,Bang Khen,5,10220,บางเขน,189539,2,13.873889,100.596389
4,Bang Kho Laem,31,10120,บางคอแหลม,94956,3,13.693333,100.502500


In [31]:
df = df[["Thai","Latitude","Longitude"]]
df.columns = ["District","Latitude","Longitude"]
df['City'] = "Bangkok"
df.head()

,District,Latitude,Longitude,City
0,บางบอน,13.659200,100.399100,Bangkok
1,บางกะปิ,13.765833,100.647778,Bangkok
2,บางแค,13.696111,100.409444,Bangkok
3,บางเขน,13.873889,100.596389,Bangkok
4,บางคอแหลม,13.693333,100.502500,Bangkok


In [32]:
bangkok_data = df.copy()

In [35]:
bangkok_data.District.dropna(inplace=True)
bangkok_data.shape

(50, 4)

In [36]:
from geopy.geocoders import Nominatim
address = 'Bangkok'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangkok are {}, {}.'.format(latitude, longitude))

/Users/rachataburin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Bangkok are 13.7542529, 100.493087.


In [40]:
import folium
map_bangkok= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, city, district in zip(bangkok_data['Latitude'], bangkok_data['Longitude'], bangkok_data['City'], bangkok_data['District']):
    label = '{}, {}'.format( district, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangkok)  
    
map_bangkok

In [41]:
CLIENT_ID = '2JE0DC5F1TTTL0DOPAF35FY0QBBVO1KN2TUKQPM5GEOWBZIU' # your Foursquare ID
CLIENT_SECRET = 'UQMVUMVLC3CDZFEGI2GCE12STFFIKQE2C40B0UAKJBAJJLTD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2JE0DC5F1TTTL0DOPAF35FY0QBBVO1KN2TUKQPM5GEOWBZIU
CLIENT_SECRET:UQMVUMVLC3CDZFEGI2GCE12STFFIKQE2C40B0UAKJBAJJLTD
